In [1]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import Binarizer

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import auc, mean_squared_error, mean_squared_log_error, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, label_binarize
from sklearn.ensemble     import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree         import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.impute       import SimpleImputer
# ---- 추가 모델
from sklearn.ensemble     import AdaBoostClassifier, VotingClassifier
from xgboost              import XGBClassifier
from lightgbm             import LGBMClassifier
from catboost             import CatBoostClassifier # -> 카테고리성에서 아주 좋은 성능

from imblearn.over_sampling import ADASYN


In [3]:
df=pd.read_csv('./train.csv')
test=pd.read_csv('./test.csv')

In [4]:
df=df.drop('ID',axis=1)

test=test.drop('ID',axis=1)

In [5]:
import imblearn
from imblearn.over_sampling import SMOTE, ADASYN,RandomOverSampler
from imblearn.combine import SMOTETomek

# X_, y_ = SMOTE(random_state=111,sampling_strategy=0.4).fit_resample(df, df['전화해지여부'])
# y_.value_counts()
# df_over = pd.concat([X_,y_],axis=1)

# df_over, y_ = ADASYN(random_state=0,sampling_strategy=0.6,n_neighbors=3).fit_resample(df, df['전화해지여부'])
# print(y_[y_==0].shape,y_[y_==1].shape)

# df, y_ = SMOTETomek(random_state=0,sampling_strategy=0.8).fit_resample(df, df['전화해지여부'])
# print(y_[y_==0].shape,y_[y_==1].shape)

df, y_ = RandomOverSampler(random_state=0,sampling_strategy=0.5).fit_resample(df, df['전화해지여부'])
print(y_[y_==0].shape,y_[y_==1].shape)

print(df.shape)
df.head(2)

(26882,) (13441,)
(40323, 13)


,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,전화해지여부
0,329,0,99.2,93,27.3,268.8,68,28.92,262.9,328,32.89,2,0
1,2,80,323.9,323,83.7,269.4,326,32.09,322.8,209,32.32,2,0


In [6]:
# df['주간시간당요금']=np.where(df['주간통화시간']==0,df['주간통화요금'],df['주간통화요금']/df['주간통화시간'])
# df['저녁시간당요금']=np.where(df['저녁통화시간']==0,df['저녁통화요금'],df['저녁통화요금']/df['저녁통화시간'])
# df['밤시간당요금']=np.where(df['밤통화시간']==0,df['밤통화요금'],df['밤통화요금']/df['밤통화시간'])
# df['가입월'] = round(np.floor(df['가입일'] / 30) - (  ( np.floor(df['가입일'] / 30) / 12).astype(int) * 12  ),4)
# df['총통화요금'] = df['주간통화요금']+ df['저녁통화요금']+df['밤통화요금']
# df['월별요금']=df['총통화요금'] /(df['가입월']+1)

# test['월별요금']=test['총통화요금'] /(test['가입월']+1)
# test['총통화요금'] = test['주간통화요금']+ test['저녁통화요금']+test['밤통화요금']
# test['가입월'] = round(np.floor(test['가입일'] / 30) - (  ( np.floor(test['가입일'] / 30) / 12).astype(int) * 12  ),4)
# test['밤시간당요금']=np.where(test['밤통화시간']==0,test['밤통화요금'],test['밤통화요금']/test['밤통화시간'])
# test['저녁시간당요금']=np.where(test['저녁통화시간']==0,test['저녁통화요금'],test['저녁통화요금']/test['저녁통화시간'])
# test['주간시간당요금']=np.where(test['주간통화시간']==0,test['주간통화요금'],test['주간통화요금']/test['주간통화시간'])

# df = df.drop(['주간통화요금','저녁통화요금','밤통화요금','총통화요금'],axis=1)
# test = test.drop(['주간통화요금','저녁통화요금','밤통화요금','총통화요금'],axis=1)

In [7]:
df.columns

Index(['가입일', '음성사서함이용', '주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수',
       '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금', '상담전화건수', '전화해지여부'],
      dtype='object')

In [8]:
X = df.drop(['전화해지여부'], axis = 1)
y = df['전화해지여부']
# model = ExtraTreesClassifier(max_depth=60, n_estimators=200, random_state=1)
# model = XGBClassifier(max_depth=10, n_estimators= 300, random_state=1)
model = RandomForestClassifier(max_depth=100, min_samples_leaf=1, n_estimators=200, random_state=1)

X_train,X_test, y_train, y_test = train_test_split(X,y,random_state = 1,test_size=0.2)
model.fit(X_train,y_train)
pre=model.predict(X_test)
score=accuracy_score(y_test, pre)

print(f'f1_score : {f1_score(y_test,pre):.5}')
print(f'정밀도 : {precision_score(y_test,pre):.5}')
print(f'재현율 : {recall_score(y_test,pre):.5}')
print(f'accuracy_score : {score:.5}')

f1_score : 0.97003
정밀도 : 0.9533
재현율 : 0.98735
accuracy_score : 0.97967


In [9]:
proba = model.predict_proba(X_test)

In [10]:
proba_c1 = proba[:,1]

th_list = [0.35, .45 , .5]
for th in th_list:
    print(th)
    brz = Binarizer(threshold=th)
    th_pred = brz.fit_transform(proba_c1.reshape(-1,1))
    print(confusion_matrix(y_test, th_pred))

0.35
[[5176  201]
 [  24 2664]]
0.45
[[5230  147]
 [  30 2658]]
0.5
[[5247  130]
 [  34 2654]]


In [11]:
s = pd.read_csv('./sample_submission.csv')
s.head(1)

,ID,전화해지여부
0,TEST_00000,0


In [12]:
s['전화해지여부']=model.predict(test)
s.head()

,ID,전화해지여부
0,TEST_00000,0
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,1
4,TEST_00004,1


In [13]:
s.to_csv("tt.csv", index=False)

# automl 코드

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from supervised.automl import AutoML

# df = pd.read_csv(
#     "https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv",
#     skipinitialspace=True,
# )
# X_train, X_test, y_train, y_test = train_test_split(
#     df[df.columns[:-1]], df["income"], test_size=0.25
# )

# automl = AutoML()
# automl.fit(X_train, y_train)

# predictions = automl.predict(X_test)

In [16]:
test_y = pd.read_csv('./automl_가답.csv', encoding='cp949')
test_y.head()

,ID,해지여부
0,TEST_00000,0
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,1
4,TEST_00004,1
